# **Kaggle – DataTops®**
Tu TA ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a tu profe a mejorar ese modelo?

## Aspectos importantes
- Última submission:
    - Mañana: 17 de febrero a las 5pm
    - Tarde: 19 de febrero a las 5pm
- **Enlace de la competición**: https://www.kaggle.com/t/c5cc87b50c4b4770bdc8f5acbe15577d
- **Requisito**: Estar registrado en [Kaggle](https://www.kaggle.com/)

## Métrica:
El error cuadrático medio (RMSE, por sus siglas en inglés) es una medida de la desviación estándar de los residuos (errores de predicción). Los residuos representan la diferencia entre los valores observados y los valores predichos por el modelo. El RMSE indica qué tan dispersos están estos errores: cuanto menor es el RMSE, más cercanas están las predicciones a los valores reales. En otras palabras, el RMSE mide qué tan bien se ajusta la línea de regresión a los datos.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## 1. Librerías

In [53]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import urllib.request
import re  # (estándar de Python) necesario para extraer datos desde texto

## 2. Datos

In [54]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle
df = pd.read_csv("./data/train.csv")

In [55]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
sample_sub = pd.read_csv("./data/sample_submission.csv")

print("train:", train.shape)
print("test:", test.shape)
print("sample:", sample_sub.shape)

train.head()


train: (912, 13)
test: (391, 12)
sample: (391, 2)


,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
0,755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
1,618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
2,909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
3,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
4,286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


### 2.1 Exploración de los datos

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_in_euros    912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [57]:
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
0,755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
1,618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
2,909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
3,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
4,286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [58]:
df.tail()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
907,28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
908,1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
909,78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
910,23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
911,229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [59]:
df.describe()

,laptop_ID,Inches,Price_in_euros
count,912.000000,912.000000,912.000000
mean,650.312500,14.981579,1111.724090
std,382.727748,1.436719,687.959172
min,2.000000,10.100000,174.000000
25%,324.750000,14.000000,589.000000
50%,636.500000,15.600000,978.000000
75%,982.250000,15.600000,1483.942500
max,1320.000000,18.400000,6099.000000


In [60]:
train.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
laptop_ID,912.0,NaN,NaN,NaN,650.3125,382.727748,2.0,324.75,636.5,982.25,1320.0
Company,912,19,Lenovo,202,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Product,912,480,XPS 13,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TypeName,912,6,Notebook,509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Inches,912.0,NaN,NaN,NaN,14.981579,1.436719,10.1,14.0,15.6,15.6,18.4
ScreenResolution,912,36,Full HD 1920x1080,349,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cpu,912,107,Intel Core i5 7200U 2.5GHz,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ram,912,9,8GB,434,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Memory,912,37,256GB SSD,282,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gpu,912,93,Intel HD Graphics 620,185,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
train.isna().sum().sort_values(ascending=False).head(20)


laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price_in_euros      0
dtype: int64

In [62]:
# Función de Feature Engineering
def feature_engineering(df):
    df = df.copy()

    # RAM: "8GB" -> 8
    df["Ram_GB"] = df["Ram"].str.replace("GB", "", regex=False).astype(float)

    # Weight: "1.86kg" -> 1.86
    df["Weight_kg"] = df["Weight"].str.replace("kg", "", regex=False).astype(float)

    # ScreenResolution: extraer ancho y alto (ej. 1920x1080)
    wh = df["ScreenResolution"].str.extract(r'(?P<width>\d+)\s*x\s*(?P<height>\d+)')
    df["Screen_Width"] = wh["width"].astype(float)
    df["Screen_Height"] = wh["height"].astype(float)

    # PPI (pixels per inch)
    df["PPI"] = np.sqrt(df["Screen_Width"]**2 + df["Screen_Height"]**2) / df["Inches"]

    # Flags de pantalla
    df["Touchscreen"] = df["ScreenResolution"].str.contains("Touchscreen", case=False, na=False).astype(int)
    df["IPS_Panel"] = df["ScreenResolution"].str.contains("IPS", case=False, na=False).astype(int)

    # CPU: marca y GHz
    df["Cpu_Brand"] = df["Cpu"].str.split().str[0]
    df["Cpu_GHz"] = df["Cpu"].str.extract(r'(\d+(\.\d+)?)\s*GHz')[0].astype(float)

    # GPU: marca
    df["Gpu_Brand"] = df["Gpu"].str.split().str[0]

    # Memory: desglose de almacenamiento
    def parse_memory(mem):
        parts = [p.strip() for p in str(mem).split("+")]
        out = {"SSD_GB": 0.0, "HDD_GB": 0.0, "Flash_GB": 0.0, "Hybrid_GB": 0.0}

        for p in parts:
            m = re.search(r"(\d+(\.\d+)?)\s*(TB|GB)", p)
            if not m:
                continue

            size = float(m.group(1))
            unit = m.group(3)
            gb = size * 1024 if unit == "TB" else size

            pl = p.lower()
            if "ssd" in pl:
                out["SSD_GB"] += gb
            elif "hdd" in pl:
                out["HDD_GB"] += gb
            elif "flash" in pl:
                out["Flash_GB"] += gb
            elif "hybrid" in pl:
                out["Hybrid_GB"] += gb
            else:
                out["HDD_GB"] += gb

        out["TotalStorage_GB"] = out["SSD_GB"] + out["HDD_GB"] + out["Flash_GB"] + out["Hybrid_GB"]
        return pd.Series(out)

    mem_feats = df["Memory"].apply(parse_memory)
    df = pd.concat([df, mem_feats], axis=1)

    return df


In [63]:
#Separar X/y y aplicar FE
target = "Price_in_euros"

X = train.drop(columns=[target])
y = train[target].copy()

X_fe = feature_engineering(X)
test_fe = feature_engineering(test)

X_fe.head()


,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,...,Touchscreen,IPS_Panel,Cpu_Brand,Cpu_GHz,Gpu_Brand,SSD_GB,HDD_GB,Flash_GB,Hybrid_GB,TotalStorage_GB
0,755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,...,0,0,Intel,2.0,Intel,256.0,0.0,0.0,0.0,256.0
1,618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,...,0,0,Intel,2.6,Nvidia,0.0,1024.0,0.0,0.0,1024.0
2,909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,...,0,0,Intel,2.7,Nvidia,0.0,1024.0,0.0,0.0,1024.0
3,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,...,0,0,Intel,1.8,Intel,0.0,0.0,128.0,0.0,128.0
4,286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,...,0,0,Intel,2.0,AMD,0.0,1024.0,0.0,0.0,1024.0


In [64]:
# Datos originales
X_original = train.drop(columns=[target])
y_target = train[target].copy()

# Feature Engineering aplicado
X_features = feature_engineering(X_original)
test_features = feature_engineering(test)



### 2.3 Definir X e y

In [65]:
target = "Price_in_euros"

# Datos “raw”
X_raw = train.drop(columns=[target])
y_target = train[target].copy()

# Feature engineering (NO borramos columnas originales)
X_processed = feature_engineering(X_raw)
test_processed = feature_engineering(test)

X_processed.head()


,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,...,Touchscreen,IPS_Panel,Cpu_Brand,Cpu_GHz,Gpu_Brand,SSD_GB,HDD_GB,Flash_GB,Hybrid_GB,TotalStorage_GB
0,755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,...,0,0,Intel,2.0,Intel,256.0,0.0,0.0,0.0,256.0
1,618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,...,0,0,Intel,2.6,Nvidia,0.0,1024.0,0.0,0.0,1024.0
2,909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,...,0,0,Intel,2.7,Nvidia,0.0,1024.0,0.0,0.0,1024.0
3,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,...,0,0,Intel,1.8,Intel,0.0,0.0,128.0,0.0,128.0
4,286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,...,0,0,Intel,2.0,AMD,0.0,1024.0,0.0,0.0,1024.0


In [66]:
X = df.drop(['Price_in_euros'], axis=1)
y = df['Price_in_euros'].copy()
X.shape

(912, 12)

In [67]:
y.shape

(912,)

In [68]:
# One-Hot Encoding (convertir texto a números) + alinear columnas
X_encoded = pd.get_dummies(X_processed, drop_first=True)
test_encoded = pd.get_dummies(test_processed, drop_first=True)

# Alinear columnas para que train y test tengan EXACTAMENTE las mismas
X_encoded, test_encoded = X_encoded.align(test_encoded, join="left", axis=1, fill_value=0)

print("X_encoded:", X_encoded.shape)
print("test_encoded:", test_encoded.shape)


X_encoded: (912, 969)
test_encoded: (391, 969)


### 2.4 Dividir X_train, X_test, y_train, y_test

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X_encoded, y_target, test_size=0.2, random_state=42
)


In [70]:
X_train

,laptop_ID,Inches,Ram_GB,Weight_kg,Screen_Width,Screen_Height,PPI,Touchscreen,IPS_Panel,Cpu_GHz,...,Weight_4.36kg,Weight_4.3kg,Weight_4.42kg,Weight_4.4kg,Weight_4.5kg,Weight_4.6kg,Weight_4.7kg,Cpu_Brand_Intel,Gpu_Brand_Intel,Gpu_Brand_Nvidia
25,1118,17.3,8.0,3.00,1920.0,1080.0,127.335675,0,1,2.6,...,False,False,False,False,False,False,False,True,False,False
84,153,15.6,16.0,2.56,1920.0,1080.0,141.211998,0,0,2.8,...,False,False,False,False,False,False,False,True,False,True
10,275,13.3,8.0,1.37,2560.0,1600.0,226.983005,0,1,2.9,...,False,False,False,False,False,False,False,True,True,False
342,1100,14.0,4.0,1.54,1920.0,1080.0,157.350512,0,0,2.3,...,False,False,False,False,False,False,False,True,True,False
890,131,17.3,16.0,2.80,1920.0,1080.0,127.335675,0,0,1.8,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,578,14.0,8.0,1.94,1366.0,768.0,111.935204,0,0,1.6,...,False,False,False,False,False,False,False,True,True,False
270,996,15.6,6.0,2.20,1920.0,1080.0,141.211998,0,0,3.6,...,False,False,False,False,False,False,False,False,False,False
860,770,12.5,16.0,1.18,1920.0,1080.0,176.232574,0,0,2.8,...,False,False,False,False,False,False,False,True,True,False
435,407,15.6,4.0,2.20,1366.0,768.0,100.454670,0,0,1.1,...,False,False,False,False,False,False,False,True,True,False


In [71]:
y_train

25     2899.00
84     1249.26
10     1958.90
342    1030.99
890    1396.00
        ...   
106     389.00
270     549.00
860    1859.00
435     306.00
102    1943.00
Name: Price_in_euros, Length: 729, dtype: float64

## 3. Procesado de datos

Nuestro target es la columna `Price_in_euros`

-----------------------------------------------------------------------------------------------------------------

In [72]:
# 🎯 Nuestro target es la columna Price_in_euros
target = "Price_in_euros"

## 4. Modelado

### 4.1 Baseline de modelos


In [73]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

rf_model = RandomForestRegressor(
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)

# Entrenar modelo
rf_model.fit(X_train, y_train)

# Predicciones
predictions = rf_model.predict(X_valid)

# Calcular RMSE
rmse = root_mean_squared_error(y_valid, predictions)

rmse



333.3977855959915

In [74]:
def fit_standardize(X_df):
    mean = X_df.mean(axis=0)
    std = X_df.std(axis=0).replace(0, 1)  # evita división por 0
    X_scaled = (X_df - mean) / std
    return X_scaled, mean, std

def apply_standardize(X_df, mean, std):
    return (X_df - mean) / std


In [75]:
X_train_scaled, mean_cols, std_cols = fit_standardize(X_train)
X_valid_scaled = apply_standardize(X_valid, mean_cols, std_cols)


In [76]:
from sklearn.svm import SVR

svr_model = SVR(C=10, epsilon=0.1, kernel="rbf")

svr_model.fit(X_train_scaled, y_train)
pred_valid_svr = svr_model.predict(X_valid_scaled)

rmse_svr = root_mean_squared_error(y_valid, pred_valid_svr)
rmse_svr


742.9754934908505

In [77]:
# Entrenar FINAL con TODO el train y predecir test
rf_final = RandomForestRegressor(
    n_estimators=800,
    random_state=42,
    n_jobs=-1
)

rf_final.fit(X_encoded, y_target)
test_predictions = rf_final.predict(test_encoded)


In [78]:
submission = sample_sub.copy()
price_col = [c for c in submission.columns if "price" in c.lower()][0]

submission[price_col] = test_predictions
submission.head()


,laptop_ID,Price_in_euros
0,209,1699.863275
1,1281,298.188950
2,1168,385.941675
3,1231,1056.040200
4,1020,1071.235200


### 4.2 Sacar métricas, valorar los modelos

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

In [79]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

def evaluar_modelo(nombre, y_real, y_pred):

    rmse = root_mean_squared_error(y_real, y_pred)
    mae = mean_absolute_error(y_real, y_pred)
    r2 = r2_score(y_real, y_pred)

    print(f"Modelo: {nombre}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R2: {r2:.4f}")
    print("---------------------")


In [80]:
from sklearn.metrics import root_mean_squared_error

rmse = root_mean_squared_error(y_valid, predictions)
rmse


333.3977855959915

### 4.3 Optimización (up to you 🫰🏻)

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

## 1. Carga los datos de `test.csv` para predecir.


In [81]:
X_pred = pd.read_csv("./data/test.csv")
X_pred.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,209,Lenovo,Legion Y520-15IKBN,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,No OS,2.4kg
1,1281,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg
2,1168,Lenovo,V110-15ISK (i3-6006U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,No OS,1.9kg
3,1231,Dell,Inspiron 7579,2 in 1 Convertible,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.191kg
4,1020,HP,ProBook 640,Notebook,14.0,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.95kg


In [82]:
X_pred.tail()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
386,820,MSI,GE72MVR 7RG,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.9kg
387,948,Toshiba,Tecra Z40-C-12X,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.47kg
388,483,Dell,Precision M5520,Workstation,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia Quadro M1200,Windows 10,1.78kg
389,1017,HP,Probook 440,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,Intel HD Graphics 620,Windows 10,1.64kg
390,421,Asus,ZenBook Flip,2 in 1 Convertible,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.27kg


In [83]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         391 non-null    int64  
 1   Company           391 non-null    object 
 2   Product           391 non-null    object 
 3   TypeName          391 non-null    object 
 4   Inches            391 non-null    float64
 5   ScreenResolution  391 non-null    object 
 6   Cpu               391 non-null    object 
 7   Ram               391 non-null    object 
 8   Memory            391 non-null    object 
 9   Gpu               391 non-null    object 
 10  OpSys             391 non-null    object 
 11  Weight            391 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 36.8+ KB


 ## 2. Replicar el procesado para ``test.csv``

In [84]:
X_pred

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
0,209,Lenovo,Legion Y520-15IKBN,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,No OS,2.4kg
1,1281,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg
2,1168,Lenovo,V110-15ISK (i3-6006U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,No OS,1.9kg
3,1231,Dell,Inspiron 7579,2 in 1 Convertible,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.191kg
4,1020,HP,ProBook 640,Notebook,14.0,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.95kg
...,...,...,...,...,...,...,...,...,...,...,...,...
386,820,MSI,GE72MVR 7RG,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.9kg
387,948,Toshiba,Tecra Z40-C-12X,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.47kg
388,483,Dell,Precision M5520,Workstation,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia Quadro M1200,Windows 10,1.78kg
389,1017,HP,Probook 440,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,Intel HD Graphics 620,Windows 10,1.64kg


**¡OJO! ¿Por qué me da error?**

IMPORTANTE:

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO **BORRAR FILAS**, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER **391 FILAS**, SI O SI

**Entonces, si al cargar los datos de ``train.csv`` usaste `index_col=0`, ¿tendré que hacer lo también para el `test.csv`?**

In [85]:
# ¿Qué opináis?
# ¿Sí, no?

![wow.jpeg](attachment:wow.jpeg)

## 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`.

In [86]:
sample = sample_sub.copy()
print("sample shape:", sample.shape)
print("sample cols:", list(sample.columns))
sample.head()


sample shape: (391, 2)
sample cols: ['laptop_ID', 'Price_in_euros']


,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6


In [87]:
print("test_encoded shape:", test_encoded.shape)


test_encoded shape: (391, 969)


In [88]:
# (solo si aún no lo habías entrenado)
rf_model.fit(X_encoded, y_target)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",500
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [89]:
predictions = rf_model.predict(test_encoded)
print("len(predictions):", len(predictions))


len(predictions): 391


In [90]:
ids_ok = submission["laptop_ID"].equals(sample["laptop_ID"])
print("IDs en el mismo orden:", ids_ok)


IDs en el mismo orden: True


In [91]:
# Creamos un df con laptop_ID + predicciones
tmp_pred = pd.DataFrame({
    "laptop_ID": test["laptop_ID"],     # id del test original
    price_col: predictions
})

# Reordenar tmp_pred para que siga el orden exacto del sample
submission = sample[["laptop_ID"]].merge(tmp_pred, on="laptop_ID", how="left")

print("IDs en el mismo orden:", submission["laptop_ID"].equals(sample["laptop_ID"]))
print("submission shape:", submission.shape)
submission.head()


IDs en el mismo orden: True
submission shape: (391, 2)


,laptop_ID,Price_in_euros
0,209,1716.11618
1,1281,299.75170
2,1168,385.52882
3,1231,1040.53390
4,1020,1057.54468


In [92]:
sample = pd.read_csv("data/sample_submission.csv")

In [93]:
sample.head()

,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6


In [94]:
sample.shape

(391, 2)

## 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [95]:
# 1) Asegura que usamos el sample correcto (el CSV original)
sample = sample_sub.copy()

# 2) Asegura que predictions tiene 391 valores (predicción sobre test)
predictions = rf_model.predict(test_encoded)   # <-- MUY IMPORTANTE: test_encoded, no X_valid

# 3) Construir submission con la misma estructura que sample
submission = sample.copy()

# detectar la columna del precio automáticamente
price_col = [c for c in submission.columns if "price" in c.lower()][0]

# meter predicciones
submission[price_col] = predictions

# comprobar shapes
print("predictions length:", len(predictions))
print("submission shape:", submission.shape)
print("sample shape:", sample.shape)

submission.head()


predictions length: 391
submission shape: (391, 2)
sample shape: (391, 2)


,laptop_ID,Price_in_euros
0,209,1716.11618
1,1281,299.75170
2,1168,385.52882
3,1231,1040.53390
4,1020,1057.54468


In [106]:
print("submission shape:", submission.shape)
print("sample shape:", sample.shape)


submission shape: (0, 0)
sample shape: (391, 2)


In [107]:
#¿Cómo creamos la submission?
submission = pd.DataFrame()

In [98]:
submission.head()

""


In [99]:
submission.shape

(0, 0)

## 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [104]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.

    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.

    Si no, LEE EL MENSAJE Y HAZLE CASO.

    Si aún no:
    - apaga tu ordenador,
    - date una vuelta,
    - enciendelo otra vez,
    - abre este notebook y
    - leelo todo de nuevo.
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                df_to_submit.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")
                img = Image.open("gfg.png")
                img.show()
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [105]:
chequeador(submission)

Check the number of rows and/or columns and try again

Mensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.
